In [10]:
import pandas as pd
import numpy as np
import os, sys
# data_folder = '/Users/a80099928/PycharmProjects/dacon_star2_data/'
data_folder = 'data/'
# pd.options.display.max_rows=1000
pd.set_option('display.max_rows', 1000)

In [2]:
train_df = pd.read_csv(os.path.join(data_folder,'train.csv'))

In [11]:
train_df.head()

,game_id,winner,time,player,species,event,event_contents
0,0,1,0.00,0,T,Camera,"at (145.25, 21.5078125)"
1,0,1,0.00,1,T,Camera,"at (22.75, 147.0078125)"
2,0,1,0.02,0,T,Selection,['OrbitalCommand [3080001]']
3,0,1,0.02,0,T,Ability,(1360) - TrainSCV
4,0,1,0.14,0,T,Camera,"at (142.99609375, 24.50390625)"


In [4]:
pd.unique(train_df['game_id']).shape

(38872,)

In [5]:
len(train_df)

67091776

In [6]:
train_df.describe()

,game_id,winner,time,player
count,6.709178e+07,6.709178e+07,6.709178e+07,6.709178e+07
mean,1.944981e+04,5.012076e-01,4.277912e+00,4.992699e-01
std,1.122980e+04,4.999985e-01,2.709741e+00,4.999995e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,9.718000e+03,0.000000e+00,2.120000e+00,0.000000e+00
50%,1.948500e+04,1.000000e+00,4.150000e+00,0.000000e+00
75%,2.919700e+04,1.000000e+00,6.360000e+00,1.000000e+00
max,3.887100e+04,1.000000e+00,1.059000e+01,1.000000e+00


In [7]:
train_df.dtypes

game_id             int64
winner              int64
time              float64
player              int64
species            object
event              object
event_contents     object
dtype: object

In [27]:
# 각 게임의 첫 시작은 무조건 camera일까?
train_df[(train_df['time']==0) & (train_df['event']=='Camera')]

,game_id,winner,time,player,species,event,event_contents
0,0,1,0.0,0,T,Camera,"at (145.25, 21.5078125)"
1,0,1,0.0,1,T,Camera,"at (22.75, 147.0078125)"
1140,1,1,0.0,1,T,Camera,"at (28.0, 18.5078125)"
1141,1,1,0.0,0,P,Camera,"at (140.0, 162.0078125)"
3574,2,0,0.0,1,Z,Camera,"at (24.75, 128.0078125)"
...,...,...,...,...,...,...,...
67085605,38869,0,0.0,1,Z,Camera,"at (24.75, 128.01171875)"
67087669,38870,1,0.0,1,P,Camera,"at (31.75, 22.5078125)"
67087670,38870,1,0.0,0,P,Camera,"at (152.25, 142.0078125)"
67088668,38871,0,0.0,1,T,Camera,"at (36.75, 122.0078125)"


In [8]:
# event별로 나눠서 파일 각각 저장
import time
for event in ['Ability','AddToControlGroup','Camera','ControlGroup','GetControlGroup','Right Click','Selection','SetControlGroup']:
    st_t = time.time()
    tmp_df = train_df[train_df['event']==event]
    print(len(tmp_df))
    tmp_df.to_csv(os.path.join(data_folder,f'train_{event}.csv'),index=False)
    print(time.time()-st_t, event)

3798390
29.54979109764099 Ability
149340
7.646019697189331 AddToControlGroup
31629407
160.66735816001892 Camera
30719
8.300530195236206 ControlGroup
10051180
50.37962985038757 GetControlGroup
11565877
77.80876111984253 Right Click
9526159
51.55662488937378 Selection
340704
12.634552717208862 SetControlGroup


In [36]:
# AddToControlGroup, ControlGroup, GetControlGroup, SetControlGroup, 
# 네 개는 event_contents가 없음
# Ability, Camera, Right Click, Selection
# event_contents 복잡도
# Ability > Right Click > Selection > Camera

In [39]:
# 승/패 LABEL csv로 따로 저장
tmp_df = train_df[['game_id','winner','player','species']].drop_duplicates()
tmp_df1 = tmp_df.pivot(index = 'game_id',columns='player',values='species').reset_index()
tmp_df2 = train_df[['game_id','winner']].drop_duplicates()
tmp_df = pd.merge(tmp_df1, tmp_df2,on='game_id')
tmp_df.to_csv(os.path.join(data_folder,'train_species_labels.csv'),index=False)

In [59]:
# 게임별, 플레이어별 각 action별 count 수 따로 csv로 저장
tmp_df = train_df[['game_id','player','event','time']].groupby(['game_id','player','event']).count().reset_index()
tmp_df['player_event'] = tmp_df['player'].astype(str) + '_'+tmp_df['event']
tmp_df.pivot(index='game_id',columns='player_event',values='time').fillna(0).reset_index().to_csv(os.path.join(data_folder,'train_action_count.csv'),index=False)

# kernel reset

In [1]:
import pandas as pd
import numpy as np
import os, sys
data_folder = '/Users/a80099928/PycharmProjects/dacon_star2_data/'
pd.options.display.max_rows=1000

In [2]:
df = pd.read_csv(os.path.join(data_folder,'train_species_labels.csv'))
df1 = pd.read_csv(os.path.join(data_folder,'train_action_count.csv'))

In [3]:
df1.head()

,game_id,0_Ability,0_AddToControlGroup,0_Camera,0_ControlGroup,0_GetControlGroup,0_Right Click,0_Selection,0_SetControlGroup,1_Ability,1_AddToControlGroup,1_Camera,1_ControlGroup,1_GetControlGroup,1_Right Click,1_Selection,1_SetControlGroup
0,0,34.0,2.0,444.0,0.0,24.0,35.0,50.0,3.0,34.0,0.0,425.0,0.0,3.0,28.0,57.0,1.0
1,1,77.0,1.0,627.0,0.0,162.0,160.0,186.0,10.0,67.0,0.0,858.0,0.0,31.0,131.0,116.0,8.0
2,2,69.0,6.0,413.0,0.0,99.0,160.0,90.0,14.0,85.0,5.0,725.0,2.0,109.0,204.0,232.0,9.0
3,3,82.0,0.0,713.0,0.0,132.0,276.0,180.0,6.0,89.0,0.0,388.0,0.0,710.0,268.0,148.0,19.0
4,4,57.0,1.0,430.0,0.0,224.0,177.0,67.0,10.0,36.0,4.0,272.0,0.0,99.0,106.0,126.0,8.0


In [4]:
df = pd.merge(df, df1, on='game_id')

In [5]:
# 플레이어 종족, event별 action count만 갖고 lightGBM 돌려보자

In [6]:
df['species0_T'] = df['0'].map(lambda x: 1 if x=='T' else 0)
df['species0_P'] = df['0'].map(lambda x: 1 if x=='P' else 0)
df['species0_Z'] = df['0'].map(lambda x: 1 if x=='Z' else 0)
df['species1_T'] = df['1'].map(lambda x: 1 if x=='T' else 0)
df['species1_P'] = df['1'].map(lambda x: 1 if x=='P' else 0)
df['species1_Z'] = df['1'].map(lambda x: 1 if x=='Z' else 0)

In [7]:
df.head()

,game_id,0,1,winner,0_Ability,0_AddToControlGroup,0_Camera,0_ControlGroup,0_GetControlGroup,0_Right Click,...,1_GetControlGroup,1_Right Click,1_Selection,1_SetControlGroup,species0_T,species0_P,species0_Z,species1_T,species1_P,species1_Z
0,0,T,T,1,34.0,2.0,444.0,0.0,24.0,35.0,...,3.0,28.0,57.0,1.0,1,0,0,1,0,0
1,1,P,T,1,77.0,1.0,627.0,0.0,162.0,160.0,...,31.0,131.0,116.0,8.0,0,1,0,1,0,0
2,2,P,Z,0,69.0,6.0,413.0,0.0,99.0,160.0,...,109.0,204.0,232.0,9.0,0,1,0,0,0,1
3,3,T,P,0,82.0,0.0,713.0,0.0,132.0,276.0,...,710.0,268.0,148.0,19.0,1,0,0,0,1,0
4,4,T,Z,0,57.0,1.0,430.0,0.0,224.0,177.0,...,99.0,106.0,126.0,8.0,1,0,0,0,0,1


In [8]:
df.columns

Index(['game_id', '0', '1', 'winner', '0_Ability', '0_AddToControlGroup',
       '0_Camera', '0_ControlGroup', '0_GetControlGroup', '0_Right Click',
       '0_Selection', '0_SetControlGroup', '1_Ability', '1_AddToControlGroup',
       '1_Camera', '1_ControlGroup', '1_GetControlGroup', '1_Right Click',
       '1_Selection', '1_SetControlGroup', 'species0_T', 'species0_P',
       'species0_Z', 'species1_T', 'species1_P', 'species1_Z'],
      dtype='object')

In [9]:
_ftr = list(set(df.columns) - set(['game_id','winner','0','1']))

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=1)
train_data, valid_data = train_test_split(train_data, test_size=0.4, random_state=2)

In [12]:
print('train_data: ',train_data.shape)
print('valid_data: ',valid_data.shape)
print('test_data: ',test_data.shape)

train_data:  (18658, 26)
valid_data:  (12439, 26)
test_data:  (7775, 26)


In [13]:
X_train, y_train = train_data[_ftr], train_data['winner']
X_valid, y_valid = valid_data[_ftr], valid_data['winner']
X_test, y_test = test_data[_ftr], test_data['winner']

In [14]:
import lightgbm as lgb
lgb_train_data = lgb.Dataset(X_train, label=y_train)
lgb_valid_data = lgb.Dataset(X_valid, label=y_valid)

/Users/a80099928/opt/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [15]:
num_threads, learning_rate = 12,0.05
params = {'boosting_type': 'gbdt', 
         'objective': 'binary',
         'num_threads' : num_threads, 'learning_rate' : learning_rate,
         'vervosity':0}
bst = lgb.train(params, lgb_train_data,
               num_boost_round = 5000,
               early_stopping_rounds=10,
               valid_sets=[lgb_valid_data],
               verbose_eval=False)

In [16]:
from sklearn.metrics import roc_auc_score
predict_test = bst.predict(X_test)

In [17]:
auc = roc_auc_score(y_test, predict_test)

In [18]:
auc # 0.6448508548486072

0.6448508548486072